In [49]:
import pandas as pd
import math

In [53]:
users_sleepy = pd.read_csv(r'C:\Users\Incognitus\Downloads\entries.csv', sep=';')

In [54]:
users_sleepy

,user,dt,Unnamed: 2
0,0,12.11.2023,NaN
1,0,13.11.2023,NaN
2,0,14.11.2023,NaN
3,0,16.11.2023,NaN
4,0,17.11.2023,NaN
...,...,...,...
52320,1998,26.12.2023,NaN
52321,1998,18.06.2024,NaN
52322,1999,04.06.2023,NaN
52323,1999,14.07.2023,NaN


In [55]:
users_filter = users_sleepy.loc[:, ['user', 'dt']]

In [56]:
users_filter.head(5)

,user,dt
0,0,12.11.2023
1,0,13.11.2023
2,0,14.11.2023
3,0,16.11.2023
4,0,17.11.2023


In [57]:
count_day_use_app = users_filter.groupby('user')['dt'].count().reset_index()
count_day_use_app.columns = ['user', 'count_day_use']
count_day_use_app.head(5)

,user,count_day_use
0,0,39
1,1,42
2,2,27
3,3,20
4,4,41


In [50]:
all_users = count_day_use_app['count_day_use'].count()
day_x = count_day_use_app.query('count_day_use > 14').count()
day_14 = count_day_use_app.query('count_day_use == 14').count()

retention_day_x = day_x / all_users * 100
retention_day_14 = day_14 / all_users * 100

print(math.ceil(abs(retention_day_14 - retention_day_x)))
print()
print(count_day_use_app['count_day_use'].count())
print(count_day_use_app.query('count_day_use < 15').count())
print(count_day_use_app.query('count_day_use > 14').count())

TypeError: cannot convert the series to <class 'float'>

In [58]:
all_users = count_day_use_app['count_day_use'].count()
day_x = count_day_use_app.query('count_day_use > 14').shape[0]
day_14 = count_day_use_app.query('count_day_use >= 14').shape[0]

retention_day_x = day_x / all_users * 100
retention_day_14 = day_14 / all_users * 100

print(math.ceil(abs(retention_day_14 - retention_day_x)))

2


In [62]:
import pandas as pd
import math

# Чтение данных
users_sleepy = pd.read_csv(r'C:\Users\Incognitus\Downloads\entries.csv', sep=';')
users_filter = users_sleepy.loc[:, ['user', 'dt']]

# Преобразуем dt в datetime
users_filter['dt'] = pd.to_datetime(users_filter['dt'], dayfirst=True)

# Находим первый день захода пользователя
first_day = users_filter.groupby('user')['dt'].min().reset_index()
first_day.columns = ['user', 'first_day']

# Объединяем с исходными заходами
df = users_filter.merge(first_day, on='user')

# Считаем день с момента первого захода
df['day_since_first'] = (df['dt'] - df['first_day']).dt.days + 1

# Количество уникальных пользователей
all_users = df['user'].nunique()

# Retention на 14-й день
day_14_retention = df[df['day_since_first'] == 14]['user'].nunique()

# Retention после 14-го дня (только платящие пользователи)
day_after_retention = df[df['day_since_first'] > 14]['user'].nunique()

# Переводим в проценты
retention_14 = day_14_retention / all_users * 100
retention_after = day_after_retention / all_users * 100

# Падение retention, округляем вверх
drop = math.ceil(retention_14 - retention_after)

print(drop)
# должно быть 18

-56
